langchain pgvector
https://python.langchain.com/docs/integrations/vectorstores/pgvector/

In [83]:
# OpenAI 임베딩 사용
import os
os.environ['OPENAI_API_KEY'] = open('API_KEY', 'r').read()

## Vector DB setting

In [84]:
from langchain_postgres import PGVector
from langchain_postgres.vectorstores import PGVector
from langchain_openai import OpenAIEmbeddings

connection=f"postgresql+psycopg://postgres@localhost:5432/my_vec_db"
collection_name = "kb_insure"

vector_store = PGVector(
    embeddings=OpenAIEmbeddings(model="text-embedding-3-large"),
    collection_name=collection_name,
    connection=connection,
    use_jsonb=True,
)

## PDF 기반 질의 응답(Question-Answering)Permalink

데이터로드

In [85]:
from langchain.document_loaders import PyPDFLoader

# 약관문서 샘플 : https://www.kbinsure.co.kr/CG802030002.ec
# 20240401 일자 버전.
# PDF 파일 로드
loader = PyPDFLoader("/Users/jaesolshin/Documents/GitHub/pg_test/data/20240401_15101_1.pdf")
document = loader.load()
document[0].page_content[:200] # 내용 추출

'KB개인상해보험'

데이터분할

In [86]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(document)

In [87]:
len(texts)

203

벡터스토어에 저장

In [88]:
vector_store.add_documents(texts)

['6b022c37-6a3c-4ef7-bd0e-53a2dc11e6ba',
 'ab6d5cae-70c9-4b5d-9770-f0d81d742417',
 '521430ce-139d-4192-ab55-8d3b05dba37a',
 '2343bdf0-02fb-486a-b536-390626545f1e',
 '0c33ba80-aa35-4a4f-ad2d-a26538cc6941',
 '2ee9559c-e77f-441f-abd3-4e8a5eb01243',
 'eadab098-8df5-4307-a092-6b1a39a13af2',
 '27af625d-5c3d-449b-bfbe-c881134b585f',
 'bf2ab40c-bf2a-4079-9c6f-224c7efcd6f0',
 '4aea0f72-17a0-4f65-901e-ff06422e7204',
 '88f4ac68-5575-446d-85b9-84a2fd742152',
 'd9d5baa8-ed67-4d24-9d87-7749d2ecc400',
 '05e35f72-e8eb-4723-bceb-d7e053d2ea5e',
 '49b1859c-9c26-41a3-abcc-64239b3a404a',
 'c960bf67-4750-4a91-8c6b-760e63edfc20',
 '5cc47c5e-6f7a-4f0c-bdcc-198ed261ee2d',
 '4ac1a755-83fa-4019-afff-71cee056fa6d',
 '3f259f07-272d-4e8c-9cac-5e409f2b2d6d',
 'b697e9c1-a78e-4133-9477-c87b7214eb0b',
 '7d88f9c9-063a-426c-a89f-e8b9c02901b7',
 'dc87327e-ea3c-4658-8dc5-5a030855fd3b',
 '6ea7a0d8-4ddd-4042-a213-d0c39e941737',
 '39af4fd1-e11b-4954-be7d-7c77c0b574be',
 'c640465a-9d0e-472b-ace3-e23d8ddd9f11',
 'ce191f28-2364-

## 벡터스토어 검색

In [89]:
vector_store.similarity_search(query="보험금의 지급사유",k=3)

[Document(id='49b1859c-9c26-41a3-abcc-64239b3a404a', metadata={'page': 13, 'source': '/Users/jaesolshin/Documents/GitHub/pg_test/data/20240401_15101_1.pdf'}, page_content='- 3 -\n의하거나 또는 그 이전에 발생한 후유장해를 포함합니다), 후유장해보험금이 지급되지 않았던 피보험자에게 그 신체의 동일 부위에 또다시 제8항에 규정하는 후유장해상태가 발생하였을 경우에는 직전까지의 후유장해에 대한 후유장해보험금이 지급된 것으로 보고 최종 후유장해 상태에 해당되는 후유장해보험금에서 이를 차감하여 지급합니다.【사 례】이 계약의 보장개시전의 원인에 의하거나 또는 그 이전에 발생한 장해로 후유장해보험금의 지급사유가 되지 않았던 장해 :보험가입 전 한 팔의 손목관절에 심한 장해(지급률 20%)가 있었던 피보험자가 보험가입 후 상해로 그 손목관절에 기능을 완전히 잃은 경우(지급률 30%)에는 보험가입 후 발생한 상해로 인한 장해지급률 30%에서 보험가입 전 발생한 장해지급률 20%를 차감한 10%에 해당하는 후유장해보험금을 지급⑩ 회사가 지급하여야 할 하나의 상해로 인한 후유장해보험금은 보험가입금액을 한도로 합니다.제5조(보험금을 지급하지 않는 사유) ① 회사는 다음 중 어느 한가지로 보험금 지급사유가 발생한 때에는 보험금을 지급하지 않습니다. 1. 피보험자가 고의로 자신을 해친 경우. 다만, 피보험자가 심신상실 등으로 자유로운 의사결정을 할 수 없는 상태에서 자신을 해친 경우에는 보험금을 지급합니다. 2. 보험수익자가 고의로 피보험자를 해친 경우. 다만, 그 보험수익자가 보험금의 일부 보험수익자인 경우에는 다른 보험수익자에 대한 보험금은 지급합니다. 3. 계약자가 고의로 피보험자를 해친 경우 4. 피보험자의 임신, 출산(제왕절개를 포함합니다), 산후기. 그러나, 회사가 보장하는 보험금 지급사유와 보장개시일부터 2년이 지난 후에 발생한 습관성

In [90]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    # search_kwargs={"k": 3, "fetch_k": 2, "lambda_mult": 0.5},
    search_kwargs={"k": 10, "fetch_k": 3, "lambda_mult": 0.5},
)
retriever.invoke("보험금의 지급사유")

[Document(id='49b1859c-9c26-41a3-abcc-64239b3a404a', metadata={'page': 13, 'source': '/Users/jaesolshin/Documents/GitHub/pg_test/data/20240401_15101_1.pdf'}, page_content='- 3 -\n의하거나 또는 그 이전에 발생한 후유장해를 포함합니다), 후유장해보험금이 지급되지 않았던 피보험자에게 그 신체의 동일 부위에 또다시 제8항에 규정하는 후유장해상태가 발생하였을 경우에는 직전까지의 후유장해에 대한 후유장해보험금이 지급된 것으로 보고 최종 후유장해 상태에 해당되는 후유장해보험금에서 이를 차감하여 지급합니다.【사 례】이 계약의 보장개시전의 원인에 의하거나 또는 그 이전에 발생한 장해로 후유장해보험금의 지급사유가 되지 않았던 장해 :보험가입 전 한 팔의 손목관절에 심한 장해(지급률 20%)가 있었던 피보험자가 보험가입 후 상해로 그 손목관절에 기능을 완전히 잃은 경우(지급률 30%)에는 보험가입 후 발생한 상해로 인한 장해지급률 30%에서 보험가입 전 발생한 장해지급률 20%를 차감한 10%에 해당하는 후유장해보험금을 지급⑩ 회사가 지급하여야 할 하나의 상해로 인한 후유장해보험금은 보험가입금액을 한도로 합니다.제5조(보험금을 지급하지 않는 사유) ① 회사는 다음 중 어느 한가지로 보험금 지급사유가 발생한 때에는 보험금을 지급하지 않습니다. 1. 피보험자가 고의로 자신을 해친 경우. 다만, 피보험자가 심신상실 등으로 자유로운 의사결정을 할 수 없는 상태에서 자신을 해친 경우에는 보험금을 지급합니다. 2. 보험수익자가 고의로 피보험자를 해친 경우. 다만, 그 보험수익자가 보험금의 일부 보험수익자인 경우에는 다른 보험수익자에 대한 보험금은 지급합니다. 3. 계약자가 고의로 피보험자를 해친 경우 4. 피보험자의 임신, 출산(제왕절개를 포함합니다), 산후기. 그러나, 회사가 보장하는 보험금 지급사유와 보장개시일부터 2년이 지난 후에 발생한 습관성

## LLM 정의

In [91]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

### 프롬프트 템플릿

In [114]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import SystemMessage

system_prompt = SystemMessage(content=
        """당신은 전문 상담원입니다. 아래 지침에 따라 사용자의 질문에 답변을 제공하세요.
        ---------------------
        1. 주어진 정보만 활용하여 답변을 제공하세요. 주어진 정보로 답변을 할 수 없는 경우, 정중하게 답변을 제공할 수 없다고 설명합니다.
        2. 답변은 정제된 형식과 문어체로 작성하며, 친절하고 자세한 내용을 제공합니다.
        ---------------------
        """
)

template = (
        """다음은 문맥 정보입니다.\n"
        ---------------------\n
        {context}"
        \n---------------------\n
        문맥 정보를 고려하여 {query}에 가장 적합한 부분을 제공하세요.
        최종 출력에는 제목을 포함하지 마세요."""
)

prompt = ChatPromptTemplate.from_messages([system_prompt, template])

### Langchain 생성

In [115]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
        {"context": retriever | format_docs, "query": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

### 테스트

In [116]:
chain.invoke("보험금의 지급사유")

'주어진 문맥 정보에 따르면, 보험금의 지급사유는 다음과 같습니다:\n\n보험금은 피보험자가 보험계약에 명시된 사고나 질병으로 인해 후유장해 상태가 발생한 경우 지급됩니다. 후유장해보험금은 피보험자의 신체 동일 부위에 발생한 후유장해 상태에 대해 지급되며, 이전에 발생한 후유장해에 대한 보험금이 지급되지 않았던 경우에도 동일 부위에 새로운 후유장해가 발생하면 직전까지의 후유장해에 대한 보험금이 지급된 것으로 간주하여 최종 후유장해 상태에 해당하는 보험금에서 차감하여 지급됩니다.\n\n또한, 보험금 지급사유는 피보험자가 고의로 자신을 해친 경우를 제외하고, 심신상실 등으로 자유로운 의사결정을 할 수 없는 상태에서 자신을 해친 경우에는 보험금을 지급합니다. 보험수익자가 고의로 피보험자를 해친 경우에도 보험금이 지급되지 않으며, 피보험자의 임신, 출산, 산후기와 관련된 경우에도 보험금이 지급되지 않습니다. 다만, 보장개시일부터 2년이 지난 후 발생한 습관성 유산, 불임 및 인공수정 관련 합병증으로 인한 경우에는 보험금을 지급합니다.\n\n보험금 지급사유가 발생한 경우, 계약자 또는 피보험자나 보험수익자는 지체 없이 그 사실을 회사에 통지해야 하며, 보험수익자는 청구서, 사고증명서, 신분증 등의 서류를 제출하여 보험금을 청구해야 합니다.'

### 함수로 변경

In [117]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

def extract(retriever, system, prompt, llm, text_query):
    chain = (
            {"context": retriever | format_docs, "query": RunnablePassthrough()}
            | ChatPromptTemplate.from_messages([system, template])
            | llm
            | StrOutputParser()
        )
    output = chain.invoke(text_query)

    return output

### 테스트

In [118]:
query = "보험금의 지급사유"
extract(retriever, system_prompt, prompt, llm, query)

'주어진 문맥 정보에 따르면, 보험금의 지급사유는 다음과 같습니다:\n\n보험금은 피보험자가 보험계약에 명시된 사고나 질병으로 인해 후유장해 상태가 발생한 경우 지급됩니다. 특히, 동일한 신체 부위에 후유장해가 발생한 경우, 이전에 발생한 후유장해에 대한 보험금이 지급된 것으로 간주하고, 최종 후유장해 상태에 해당하는 보험금에서 이를 차감하여 지급합니다. 또한, 보험금은 보험가입금액을 한도로 하여 지급됩니다.\n\n보험금 지급사유가 발생한 경우, 계약자 또는 피보험자나 보험수익자는 지체 없이 그 사실을 회사에 통지해야 하며, 보험수익자는 청구서, 사고증명서, 신분증 등의 서류를 제출하여 보험금을 청구해야 합니다.'

In [119]:
query = "오늘 점심메뉴 추천"
extract(retriever, system_prompt, prompt, llm, query)

'죄송하지만 제공된 문맥 정보로는 점심 메뉴 추천에 적합한 내용을 찾을 수 없습니다. 다른 질문이 있으시면 도와드리겠습니다.'